<div class="alert alert-block alert-success"><b>Working on the 843 clean samples in H1:</b>

The 843 samples are clustering based on three sets of variables.  Four cluster analyses are executed with K=3, 4, 5, and 6 for each set of variables.
<ol>
<li>Minmax scaled v31 feature variables.</li>
<li>Minmax scaled IOA rating variables (two for each band).  Not measure value -100 is replaced by 0 before clustering.</li>
<li>Combination of the first two.</li>
</ol>

All Minmax scaling and clustering models are saved in "joblib" files. 
<ol>
<li>Two min_max scaling models, one for v31 and the other for IOA variable set.  They are saved in the ".//trained_models//scalers" folder</li>
<li>Twelve clustering models created: Four for each of the three variable sets.  They are saved in the ".//trained_models//clustering" folder</li>
</ol>

Two MongoDB collections are created.
<ul>
<li>H1_S843_clusters: cluster membership for each case under each variables set and value of K.</li>
<li>H1_S843_clusters_summary:  For each variable set each value of K, this collection stores the cluster size, cluster mean, and cluster variance.</li>
</ul>

</div>


#### A note on converting Jupyter Notebook output to MS Word Documents

- The best way to convert the ipynb file (Jupyter Notebook) to a docx file is to follow the two step approach explained in: https://blog.ouseful.info/2017/06/13/using-jupyter-notebooks-for-assessment-export-as-word-docx-extension/, i.e. run the following two commands on the anaconda command line
- Step 1: $ jupyter nbconvert --no-input --to html file_name.ipynb (use --no-input to exclude code cells, i.e. convert only markdown cells)

- Step 2: $ pandoc -s file_name.html -o file_name.docx
- This approach produces, by far, the best quality docx ouput, no distortion of either the text or the graphs.  The only drawback is that it include the hidden code cells made by the "hide all" extension.  I have to manually delete those contents from the produced docx file.
#### convert ijynb to a docx file
- https://nbconvert.readthedocs.io/en/latest/index.html
- install nbconvert [pip install nbconvert]
- install pandoc: [https://github.com/jgm/pandoc/releases/tag/3.1.3]

In [1]:
import sys

oneDrive_root={}
oneDrive_root[1]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[2]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[3]="C:\\Users\\tsaic\\OneDrive - State University of New York at New Paltz\\"  # laptop

site=3   # the short or long business OneDrive directory name
lib_dir=oneDrive_root[site]+'\\Prudentia\\Tsaipy'
# append additional library path for this study
sys.path.append(lib_dir)

In [2]:
import pandas as pd
import numpy as np
import joblib
#import sklearn as sk
#from sklearn.cluster import KMeans

from dbconnect.mongodb import cursor_to_dataframe3
from WFN_lib.WindFarm import distance2center, write_df_sub2Excel
from WFN_lib.mongodb_util import flatten_dictionary
from WFN_lib.cluster_classify import cluster_analysis
import copy

In [3]:
from pymongo import MongoClient
import pymongo
from dbconnect import mongodb as mdb

<div class="alert alert-block alert-info"><b>MongoDb Parameters:</b>

</div>

In [4]:
## pymongo connect to mongodb database and collection
db_name = "Windfarm_S6000"
client = MongoClient('localhost', 27017)  # connect to the db engine
db = client[db_name]
coll_data=db['S6000_Data']

print(f"Rows in coll_data = {coll_data.count_documents({})}")

Rows in coll_data = 6000


<div class="alert alert-block alert-warning"><b>I. The following three blocks pick the clean samples</b>

Only need to do it once.   The rest of the code dependens only on data already stored in the mongodb database.
</div>

In [5]:
## Read in the min-max normalized data
df_S972 = pd.read_excel('.//Results//cluster_allotment//Cluster_S972_MinMax31V_result.xlsx', index_col=0, header=0)
df_S130 = pd.read_excel('.//Results//cluster_allotment//More_files_to_exclude_from_S972.xlsx', sheet_name='remove_ids',index_col=None, header=0)

#df_minmax_S6000 = pd.read_excel('.//Results//data_sets//MinMax_data.xlsx', index_col=0, header=0)
print(f"df_S972 shape = {df_S972.shape};  df_S130 shape = {df_S130.shape}")

df_S972 shape = (972, 6);  df_S130 shape = (130, 1)


In [6]:
## get file ID for those need to be removed from S972
S130 = ['H1_'+str(s).zfill(4) for s in df_S130['ID']]
print(f"len(S130) = {len(S130)}")
### Find the element in S130 that is not in df_S972.index
S130_not_in_S972 = [s for s in S130 if s not in df_S972.index]
print(f"file in S130 but not in S872 = {S130_not_in_S972}")
### Get the file ID's for S972 after removal of S130
S843 = [s for s in df_S972.index if s not in S130]
print(f"len(S843) = {len(S843)}")
print(len(df_S972.index))

len(S130) = 130
file in S130 but not in S872 = ['H1_0600']
len(S843) = 843
972


In [7]:
### Create a new collection to store IDs of S843
coll_subset = 'Subset_index'
try:
    db.validate_collection(coll_subset)  # Try to validate a collection   
    db[coll_subset].drop()
    print(f"Collection (table), {coll_subset}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
    print(f"Collection, {coll_subset}, doesn't exist") 

coll_sub = db[coll_subset]
coll_sub.delete_many({'_id':'H1_S843'})
coll_sub.insert_one({'_id':'H1_S843', 'site':'H1', 'id_list':S843, 'size':len(S843), "description":"These are clean files from H1"})

Collection (table), Subset_index, dropped


<div class="alert alert-block alert-info"><b>II. Working on the clean subset of data from H1</b></div>

<div class="alert alert-block alert-success"><b>II.1 Extract data for the clean set and perform Cluster analysis</b>
<ul>
<li>The clean subset contains samples free of environmental sound contamination.</li>
<li>The minmax data is extracted from the minmax data created from the 3000 samples in H1.  Since the clean set is extracted from H1 only, we use the scaling algorithm performed on H1 set for the focused analysis on H1.</li>
<li>When calling the cluster analysis, we don't need to scale the data again.  Thus, set scaled_data=None</li>
</ul>
</div>

<div class="alert alert-bloc, alert-danger"><b>Data Type Warning!!!!</b>
The extract data has type object.  Need to chage to float64.  Its math operations work if not combined with columns of other data types, e.g. float.
When <b>mixed with other data type</b>, the object type columns do not participate in the calculation, for example, df.groupby(x).mean(), where df has columns mixed with object and float.  The operation only works on the float type columns.  This might be caused by using <b>pd.Series</b> in the next block.  Changed to just adding the values.  However, using pd.Series for IOA doesn't seem to cause the problem.  To be investigated later.
</div>

### For this selected subset S384 from H1, we use the minmax variables scaled for the 3000 files in H1.

In [8]:
### extract rows from coll_data whose 'id' is in S843v31_minmaxfeatures':1, 'label':1}})
S843 = coll_sub.find_one({'_id':'H1_S843'})['id_list']   # 'id_list' is the key for the list of file names

res = coll_data.find({'_id':{'$in':S843}},{'v31':1})

df_data_S843 = cursor_to_dataframe3(res,"_id")
print(df_data_S843.shape)
print(df_data_S843.iloc[:2,:2])

col_names=df_data_S843.columns
row_names = df_data_S843.index

### minmax normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#df_data_S843_scaled = scaler.fit_transform(df_data_S843)
#df_data_S843_scaled = pd.DataFrame(df_data_S843_scaled, columns=col_names, index=row_names)
#print(df_data_S843_scaled.iloc[:2,:2])
## save the minmax normalized data to an joblib file


(843, 31)
         v31.spectralCentroid  v31.spectralCrest
H1_0006             54.673008          28.285684
H1_0007             72.429418          28.950453


In [10]:
## Since df_data_S843_scaled is minmax scaled already, no need to scale again.  Thus, scaler_type=None
## df_S843_res is the dataframe with the cluster allotment; df_scaled is the scaled data
df_S843_cluster, df_data_S843_scaled =cluster_analysis(df_data_S843, [3,4,5,6], random_state=42, scaler_type="MinMax",n_init=10, max_iter=300, tol=0.0001,
                                save_model_dir='.//trained_models//clustering//', model_file_name='Kmeans_H1_S843_v31_MinMax_K=',
                                scaler_model_dir='.//trained_models//scalers//', scaler_model_name='_scaler_v31_H1_S843')

In [12]:
df_data_S843_scaled.head(3)

,v31.spectralCentroid,v31.spectralCrest,v31.spectralDecrease,v31.spectralEntropy,v31.spectralFlatness,v31.spectralFlux,v31.spectralKurtosis,v31.spectralRolloffPoint,v31.spectralSkewness,v31.spectralSlope,...,v31.PR,v31.Fo,v31.AMfactor,v31.DAM,v31.peakloc_unweightedSPL,v31.L63,v31.L125,v31.L250,v31.L500,v31.L1000
H1_0006,0.117261,0.675937,0.963884,0.256013,0.019118,0.261683,0.221666,0.097065,0.402864,0.497084,...,0.002544,0.555556,0.018640,0.036434,0.121951,0.539850,1.000000,0.255754,0.194950,0.086214
H1_0007,0.438056,0.737495,0.943909,0.388479,0.484214,0.225022,0.028103,0.365726,0.120473,0.817775,...,0.005157,0.444444,0.006058,0.027345,0.195122,0.391374,0.302007,0.044276,0.163008,0.128204
H1_0008,0.138865,0.916310,0.591327,0.128547,0.122270,0.268111,0.193206,0.084260,0.344021,0.596325,...,0.004277,1.000000,0.026666,0.041515,0.170732,0.480114,0.347673,0.117028,0.122350,0.199085


### Calculate Distance to Cluster Center and ranking from the closest to the farthest

In [35]:
## obtain cluster_assignment, cluster size, cluster means, cluster variances
nK=5
df_K_v31, cluster_size_v31, cluster_means_v31, cluster_vars_v31 = distance2center(df_data_S843, df_S843_cluster, [3,4,5,6])
## For specific value o K=nK
write_df_sub2Excel(df_K_v31, '', nK, './/Results_H1//S843_Results//H1_S843_K='+str(nK)+'_v31_minmax_dist2center.xlsx')
print(df_K_v31['K='+str(nK)].value_counts())

0    217
3    213
1    147
2    143
4    123
Name: K=5, dtype: int64
K=5
[0 1 2 3 4]
(217, 12)
Sort rows by:  rank_K=5
cluster = 0:  Shape of dk: (217, 12)
         K=3  K=4  K=5  K=6    dist_K=3  rank_K=3   dist_K=4  rank_K=4  \
H1_0703    2    3    0    1   52.294354         1  30.820696         1   
H1_1369    2    3    0    1   58.147479         2  34.005966         2   
H1_1528    2    3    0    1  104.920389         6  65.912693         4   

          dist_K=5  rank_K=5   dist_K=6  rank_K=6  
H1_0703  27.271821         1  42.867547         2  
H1_1369  38.396194         2  34.212291         1  
H1_1528  55.570945         3  80.902631         5  
(147, 12)
Sort rows by:  rank_K=5
cluster = 1:  Shape of dk: (147, 12)
         K=3  K=4  K=5  K=6    dist_K=3  rank_K=3    dist_K=4  rank_K=4  \
H1_1649    0    1    1    3  122.032823       120  111.436911       107   
H1_0238    0    1    1    3  103.852747       100   93.414997        90   
H1_1684    0    1    1    3  126.584311    

c:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


<div class="alert alert-block alert-success"><b>II.2 Extract IOA data for the clean subset and perform Cluster analysis</b>
<ul>
<li>The IOA assessment data is extracted from the Result_S6000 collection.</li>
<li>When calling the cluster analysis, we need to scale the data again.  Thus, set scaled_data="MinMax".</li>
<li>The minmax dataset will be used in the next analysis which combined v31_minmax with IOA_minmax data for clustering.</li>
</ul>
</div>

In [16]:
## extract rows from the Results_S6000 for IOA assessment result of 'id's in S843
res = db['S6000_Data'].find({'_id':{'$in':S843}},{'IOA_rating':1})

df_IOA_S843 = cursor_to_dataframe3(res,"_id")
print(df_IOA_S843.shape)

## drop 'IOA_rating' from column names
df_IOA_S843.columns = [col[col.find('.')+1:] for col in df_IOA_S843.columns]
## replace -100 with 0 for clustering analysis, -100 means the IOA rating is not available (does not exist since not meeting the threshold)
df_IOA_S843.replace(-100,0, inplace=True)
print(df_IOA_S843.head(3))

(843, 8)
         025_100Hz.prominence  025_100Hz.L5-L95  050_200Hz.prominence  \
H1_0006              1.157707          2.038109              1.098841   
H1_0007              1.647752          5.391628              1.380059   
H1_0008              2.281802          3.641156              2.428720   

         050_200Hz.L5-L95  100_400Hz.prominence  100_400Hz.L5-L95  \
H1_0006          1.859755              3.436457          2.442856   
H1_0007          4.596037              0.819226          3.693764   
H1_0008          3.727153              1.454394          3.026670   

         200_800Hz.prominence  200_800Hz.L5-L95  
H1_0006              2.449454          1.675939  
H1_0007              1.677567          3.083924  
H1_0008              3.555321          2.568767  


In [20]:
## IOA data has not been minmax scaled.  Need to scale it.
df_IOA_S843_res,df_IOA_S843_MinMax  =cluster_analysis(df_IOA_S843, [3,4,5,6], random_state=42, scaler_type="MinMax",n_init=10, max_iter=300, 
                                tol=0.0001, save_model_dir='.//trained_models//clustering//', model_file_name='Kmeans_H1_S843_IOA_MinMax_K=', 
                                   scaler_model_dir='.//trained_models//scalers//',scaler_model_name='_scaler_IOA_H1_S843')

In [28]:
df_K_IOA, cluster_size_IOA, cluster_means_IOA, cluster_vars_IOA = distance2center(df_IOA_S843_MinMax, df_IOA_S843_res, [3,4,5,6])
write_df_sub2Excel(df_K_IOA, '', nK, './/Results_H1//S843_Results//H1_S843_K='+str(nK)+'_IOA_dist2center.xlsx')
df_K_IOA['K='+str(nK)].value_counts()

K=5
[0 1 2 3 4]
(163, 12)
Sort rows by:  rank_K=5
cluster = 0:  Shape of dk: (163, 12)
         K=3  K=4  K=5  K=6  dist_K=3  rank_K=3  dist_K=4  rank_K=4  dist_K=5  \
H1_2420    2    3    0    5  0.124295       126  0.073612        16  0.044815   
H1_2952    2    3    0    5  0.102236        72  0.049190         3  0.048975   
H1_2392    2    3    0    5  0.129396       134  0.075060        20  0.050029   

         rank_K=5  dist_K=6  rank_K=6  
H1_2420         1  0.045734         1  
H1_2952         2  0.047144         2  
H1_2392         3  0.052831         5  
(8, 12)
Sort rows by:  rank_K=5
cluster = 1:  Shape of dk: (8, 12)
         K=3  K=4  K=5  K=6  dist_K=3  rank_K=3  dist_K=4  rank_K=4  dist_K=5  \
H1_1833    0    1    1    0  0.665399        56  0.215227         1  0.215227   
H1_2810    0    1    1    0  0.625023        54  0.255374         2  0.255374   
H1_2691    0    1    1    0  0.638926        55  0.263262         3  0.263262   

         rank_K=5  dist_K=6  rank_K=

c:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


4    385
3    214
0    163
2     73
1      8
Name: K=5, dtype: int64

<div class="alert alert-block alert-success"><b>II.3 Cluster Analysis on the extracted v31_minmax and IOA_minmax data for the clean subset</b>
<ul>
<li>Need to rescale v31_minmax within the S843</li>
<li>Concatenate df_data_S843 and df_IOA_S843_MinMax.</li>
<li>Both sets have been normalized already.  Do not need to scale variables again when calling Cluster Analysis</li>
</ul>
</div>

In [23]:
print(df_data_S843_scaled.shape)
print(df_IOA_S843_MinMax.shape)
## concatenate df_data_S843 and df_IOA_S843_MinMax on index
df_S843_combined = pd.concat([df_data_S843, df_IOA_S843_MinMax], axis=1)
print(df_S843_combined.shape)

(843, 31)
(843, 8)
(843, 39)


In [24]:
## Do not need scale the data, since both V31 and IOA have been scaled already.
df_K_combined_res, _ =cluster_analysis(df_S843_combined, [3,4,5,6], random_state=42, scaler_type=None,n_init=10, max_iter=300, tol=0.0001,
                                       save_model_dir='.//trained_models//clustering//', model_file_name='Kmeans_H1_S843_Combined_minmax_K=')
print(df_K_combined_res.head(3))

         K=3  K=4  K=5  K=6
H1_0006    0    3    4    5
H1_0007    1    0    3    1
H1_0008    0    1    4    5


In [31]:
df_K_combined, cluster_size_combined, cluster_means_combined, cluster_vars_combined = distance2center(df_S843_combined, df_K_combined_res, [3,4,5,6])
write_df_sub2Excel(df_K_combined, '', nK, './/Results_H1//S843_Results//H1_S843_K='+str(nK)+'_Combined_dist2center.xlsx')
df_K_combined['K='+str(nK)].value_counts()

K=5
[0 1 2 3 4]
(178, 12)
Sort rows by:  rank_K=5
cluster = 0:  Shape of dk: (178, 12)
         K=3  K=4  K=5  K=6    dist_K=3  rank_K=3    dist_K=4  rank_K=4  \
H1_1036    1    1    0    4  779.314779       639  560.060231        89   
H1_0085    1    1    0    4  775.601718       638  559.356637        88   
H1_2080    1    1    0    4  828.465819       654  506.829123        78   

          dist_K=5  rank_K=5    dist_K=6  rank_K=6  
H1_1036  52.685011         1  357.272866       160  
H1_0085  62.741738         2  361.149470       162  
H1_2080  65.335792         3  416.291395       179  
(32, 12)
Sort rows by:  rank_K=5
cluster = 1:  Shape of dk: (32, 12)
         K=3  K=4  K=5  K=6     dist_K=3  rank_K=3    dist_K=4  rank_K=4  \
H1_2763    0    3    1    2  1995.848257       126  764.944292        24   
H1_2893    0    3    1    2  1711.140975       123  528.621860        10   
H1_2368    0    3    1    2  1587.219559       119  532.839774        11   

           dist_K=5  rank_

c:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


3    550
0    178
4     79
1     32
2      4
Name: K=5, dtype: int64

<div class="alert alert-block alert-success"><b>II.4 Store the result to a collection, Results_S843_clusters</b> which include
<ul>
<li>The cluster allotment from the three approaches on values of K.</li>
<li>Distance to cluster center and ranking within cluster of closeness to the center.</li>
</ul>
</div>

#### Database collection for three clustering methods, v31_minmax, IOA, combination of the first two.
- One row for each case
- Columns for each method:
- cluster membership: K_values columns (one for each value of K), 
- for each K values, there are two columns, one for the distance to cluser center, one for the distance2center ranking.
- For example, if there are n different K-values, there will be n+2*n columns in this collection
- Data source:  df_K_v31, df_K_IOA, df_K_combined

In [36]:
### Create a new collection to store cluster analysis result of S843
## Drop the collection, coll_res_H1 and recreate it anew.
coll_res_cluster = 'H1_S843_clusters'
try:
    db.validate_collection(coll_res_cluster)  # Try to validate a collection   
    db[coll_res_cluster].drop()
    print(f"Collection (table), {coll_res_cluster}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {coll_res_cluster}, doesn't exist") 
     
coll_res_cluster = db[coll_res_cluster]

Collection, H1_S843_clusters, doesn't exist


In [37]:
columns=df_K_v31.columns
print(columns)
dfs = [df_K_v31, df_K_IOA, df_K_combined]
methods = ['v31_minmax', 'IOA', 'combined']
for i in range(len(df_K_v31)):
    row = dict()
    row['_id']=df_K_v31.index[i]
    for method, df in zip(methods, dfs):
        row[method] = dict()
        for col in columns:
            row[method][col]=df.iloc[i][col]
    coll_res_cluster.insert_one(row)


Index(['K=3', 'K=4', 'K=5', 'K=6', 'dist_K=3', 'rank_K=3', 'dist_K=4',
       'rank_K=4', 'dist_K=5', 'rank_K=5', 'dist_K=6', 'rank_K=6'],
      dtype='object')


<div class="alert alert-block alert-success"><b>III. Database collection for cluster summary data for the three clustering results, v31_minmax, IOA, combination of the first two:</b> 
<ol>
<li>One row for each cluster of each method</li>
<li>Columns: methods (v31_minmax, IOA, combined), K, cluster ID, cluster size (int), cluser means (31+28 columns), cluster variance (31+28 columns)</li>
<li>For cluster means and variance, v31_minmax takes the first 31 columns only, IOA takes the next 28 columns only, and combined takes all 59 columns</li>
<li>From v31_minmax: cluster_size_v31, cluster_means_v31, cluster_vars_v31</li>
<li>From IOA, cluster_size_IOA, cluster_means_IOA, cluster_vars_IOA</li>
<li>From combined: cluster_size_combined, cluster_means_combined, cluster_vars_combined</li>
</ol>
</div>

In [40]:
### Create a new collection to store cluster summary result of S843
## Drop the collection, coll_res_H1 and recreate it anew.
coll_res_cluster_summary = 'H1_S843_cluster_summary'
try:
    db.validate_collection(coll_res_cluster_summary)  # Try to validate a collection   
    db[coll_res_cluster_summary].drop()
    print(f"Collection (table), {coll_res_cluster_summary}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {coll_res_cluster_summary}, doesn't exist") 
     
coll_cluster_summary = db[coll_res_cluster_summary]

Collection (table), H1_S843_cluster_summary, dropped


<div class="alert alert-block alert-danger"><b>Setting _id in the code:</b> The rows in this documents have different columns.  v31 only has the first 31 columns, IOA has the next 28 columns, and combined has all 59 columns.  When letting mongodb insert one row at a time, there is a complaint about duplicate _id.  Normally, this column is an automatically created field, similar to the autonumber in Access.  However, in this case, I have to mannually assign _id to avoid the complaint.</div>

In [42]:
dfs = dict()
targets = ['v31_minmax', 'IOA_minmax', 'combined_minmax']
dfs['v31_minmax']=[cluster_size_v31, cluster_means_v31, cluster_vars_v31]
dfs['IOA_minmax']=[cluster_size_IOA, cluster_means_IOA, cluster_vars_IOA]
dfs['combined_minmax']=[cluster_size_combined, cluster_means_combined, cluster_vars_combined]

count=0
for target in targets:
    row = dict()
    for k in [3,4,5,6]:
        row = dict()
        row['target']=target
        row['K']=k
        ## There are k rows for each target
        row['cluster']=dict()
        for cluster in range(k):  # for each cluster for K=k means            
            row['cluster']['cluster_id']=cluster
            row['cluster']['size']=int(dfs[target][0]["K="+str(k)][cluster])  # the first dataframe, 0, is cluster_size)
            row['cluster']['cluster_mean']=dict() 
            row['cluster']['cluster_var']=dict() 
            for col1, col2 in zip(dfs[target][1]["K="+str(k)].columns, dfs[target][2]["K="+str(k)].columns):  # second dataframe is cluster means
                row['cluster']['cluster_mean'][col1]=dfs[target][1]["K="+str(k)].loc[cluster][col1]
                row['cluster']['cluster_var'][col2]=dfs[target][2]["K="+str(k)].loc[cluster][col2]
            
            row['_id']=count  # Add a unique id to the row manually
            ## move row['_id'] to the front of the dictionary
            row = {k: row[k] for k in ['_id', 'target', 'K', 'cluster']}
            print(row)
            coll_cluster_summary.insert_one(row)
            count+=1

{'_id': 0, 'target': 'v31_minmax', 'K': 3, 'cluster': {'cluster_id': 0, 'size': 361, 'cluster_mean': {'v31.spectralCentroid': 63.26066003223087, 'v31.spectralCrest': 27.59973883446138, 'v31.spectralDecrease': -59.93896126386648, 'v31.spectralEntropy': 0.15521262784596612, 'v31.spectralFlatness': 0.006650706324065491, 'v31.spectralFlux': 5.961315298115349e-06, 'v31.spectralKurtosis': 458.7670986049829, 'v31.spectralRolloffPoint': 121.58451939094942, 'v31.spectralSkewness': 14.05310400976741, 'v31.spectralSlope': -4.1675379465681515e-10, 'v31.spectralSpread': 64.38302573596148, 'v31.pitch': 399.8027252444912, 'v31.harmonicRatio': 0.012981003909070761, 'v31.LA': 39.79732943125211, 'v31.ratioLGLA': 1.5865568516275075, 'v31.ratioLCLA': 1.3927603483222168, 'v31.diffLCLA': 15.334614167337715, 'v31.peakloc': 0.6023545706371191, 'v31.peakval': 1.8462492375036523, 'v31.pos_slope': 0.8002301146330358, 'v31.neg_slope': -0.8089170381142411, 'v31.PR': 17.692340741437317, 'v31.Fo': 0.6623268698060941